# Daten importieren
*Datenquelle: [https://www.kaggle.com/datasets/amritpal333/hepatitis-c-virus-blood-biomarkers/data](https://www.kaggle.com/datasets/amritpal333/hepatitis-c-virus-blood-biomarkers/data)*

In [ ]:
import pandas as pd

# Schritt 1: Lese die entsprechende Datei ein, speichere die Daten in der Variable 'df' ab und gib sie anschließend aus.
df = ...
df

In [ ]:
# Schritt 2: Analysiere die Daten mithilfe von '.describe()'
...

In [ ]:
# Schritt 3: Führe diese Zelle aus, um für jede Spalte den Prozentsatz fehlender Werte anzuzeigen.
df.isnull().sum() / len(df)

# Daten aufteilen

In [ ]:
# Schritt 4: Weise die entsprechenden Features der Eingabevariable X und das zugehörige Ziel der Zielvariable y zu.
# Tipp: Verwende '.copy()' und '.pop()'.
X = ...
y = ...

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

# Schritt 5: Teile die Daten in Trainings- und Testdaten auf. Die Testdaten sollten 20% des Gesamtdatensatzes ausmachen und prozentual dieselbe Anzahl von Klassen wie die Trainingsdaten enthalten.
X_train, X_test, y_train, y_test = ...

# Modell-Pipeline erstellen

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Schritt 6: Führe diese Zelle aus, um die Funktion 'create_neural_network' zu definieren.
def create_neural_network(hidden_layers, hidden_layer_neurons, input_shape, hidden_layer_activation,
                 output_neurons, output_activation, optimizer, loss):
    model = tf.keras.Sequential()
    model.add(layers.Dense(units=hidden_layer_neurons, activation=hidden_layer_activation, input_shape=input_shape))

    for _ in range(hidden_layers - 1):
        model.add(layers.Dense(units=hidden_layer_neurons, activation=hidden_layer_activation))

    model.add(layers.Dense(units=output_neurons, activation=output_activation))
    model.compile(optimizer=optimizer, loss=loss)
    return model


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
!pip install scikeras==0.12.0
from scikeras.wrappers import KerasClassifier

# Schritt 7: Erstelle eine Instanz von 'StandardScaler' und vervollständige die Pipeline.
standard_scaler = ...
model = KerasClassifier(model=create_neural_network)

pipeline = Pipeline(steps=[
    ("scaler", ...),
    ("classifier", ...)
])

# Hyperparameteroptimierung

In [ ]:
from sklearn.model_selection import GridSearchCV

# Schritt 8: Erstelle jeweils eine Liste mit verschiedenen Werten für 'hidden_layers' und 'hidden_layer_neurons'.
param_grid = {
    "classifier__model__hidden_layers": [..., ...],
    "classifier__model__hidden_layer_neurons": [..., ...],
    "classifier__model__input_shape": [[13]],
    "classifier__model__hidden_layer_activation": ["relu"],
    "classifier__model__output_neurons": [1],
    "classifier__model__output_activation": ["sigmoid"],
    "classifier__model__optimizer": ["adam"],
    "classifier__model__loss": ["binary_crossentropy"],
}

# Schritt 9: Setze bei GridSearchCV die Anzahl an Folds auf 4 und scoring auf "accuracy".
grid_search = GridSearchCV(pipeline, param_grid, cv=..., scoring=...)
grid_search.fit(X_train, y_train, classifier__batch_size=512, classifier__epochs=50)

print("Best Accuracy: ", grid_search.best_score_)
print("Best Parameters: ", grid_search.best_params_)

# Early Stopping, um *epochs* besser zu bestimmen

In [ ]:
# Schritt 10: Erstelle ein Modell basierend auf den besten Werten für 'hidden_layers' und 'hidden_layer_neurons', die du im letzten Schritt gefunden hast.
early_stopping_model = create_neural_network(hidden_layers=..., hidden_layer_neurons=..., input_shape=[13], hidden_layer_activation="relu",
                 output_neurons=1, output_activation="sigmoid", optimizer="adam", loss="binary_crossentropy")

early_stopping_pipeline = Pipeline(steps=[
    ("scaler", standard_scaler),
    ("model", early_stopping_model)
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Schritt 11: Teile die Trainingsdaten wieder in neue Trainings- und Validierungsdaten auf, und führe diese Zelle aus, um das Verfahren "Early Stopping" durchzuführen.
X_train_new, X_valid, y_train_new, y_valid = train_test_split(..., ..., test_size=0.2, random_state=0)
early_stopping = EarlyStopping(min_delta=0.0001, patience=50)

standard_scaler.fit(X_train_new)

early_stopping_pipeline.fit(
    X_train_new, y_train_new,
    model__validation_data=(standard_scaler.transform(X_valid), y_valid),
    model__batch_size=512,
    model__epochs=50000,
    model__callbacks=[early_stopping]
)

# Besten Wert für *epochs* ausgeben:

In [ ]:
# Schritt 12: Bestimme mithilfe von 'early_stopping.best_epoch' die beste Anzahl an Epochen.

best_number_of_epochs = ...
best_number_of_epochs

# Ergebnisse visualisieren

In [ ]:
import seaborn as sns

history = early_stopping_pipeline.named_steps["model"].history.history

# Schritt 13: Transformiere 'history' in ein DataFrame und visualisiere diese Ergebnisse mithilfe eines Lineplots.
history_df = pd.DataFrame(...)
sns.lineplot(...)

# Finales neuronales Netz

In [ ]:
# Schritt 14: Nutze die gleichen Parameter wie in Schritt 10, um ein Modell zu erstellen, das diesmal auf den vollständigen Trainingsdaten trainiert wird.
final_model = create_neural_network(hidden_layers=..., hidden_layer_neurons=..., input_shape=[13], hidden_layer_activation="relu",
                 output_neurons=1, output_activation="sigmoid", optimizer="adam", loss="binary_crossentropy")

final_pipeline = Pipeline(steps=[
    ("scaler", standard_scaler),
    ("model", final_model)
])

# Schritt 15: Setze die Anzahl an Epochen entsprechend und führe diese Zelle aus, um das finale Modell zu trainieren.
final_pipeline.fit(X_train, y_train, model__batch_size=512, model__epochs=...)

# Finales Ergebnis (Accuracy)

In [ ]:
from sklearn.metrics import accuracy_score

y_predict_final = final_pipeline.predict(X_test) >= 0.5

#Schritt 16: Berechne die finale Accuracy basierend auf den Testdaten.
final_accuracy = accuracy_score(..., ...)

print("Final Accuracy:", final_accuracy)